In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import warnings

warnings.filterwarnings(action="ignore")

import os
import tqdm
import numpy as np
import pandas as pd

from typing import Tuple, List, Dict, Any
from constants import *
from features.user import User


# 90 of the test segments are genuine (i.e., benign) and 10 segments are entered bya masquerader (randomly sorted).

In [5]:
#   load answer for classifcation
gt_df = pd.read_csv("challengeToFill.csv", index_col=0).T[TRAIN_SEGMENT_COUNT:].reset_index(drop=True)




# dev_set_df = gt_df.T.iloc[TRAIN_SEGMENT_COUNT:, :DEV_USERS_COUNT]

In [6]:
def evaluate_user_results(pred_series: pd.Series, true_series: pd.Series):
    ''' evaluate the classification result of a single user '''
    assert len(pred_series) == len(true_series)
    assert pred_series.name == true_series.name

    equal_series = pred_series == true_series
    return equal_series.mean()

In [7]:
user_list = [f"User{i}" for i in range(10)]
data_path = [f"data/{uid}" for uid in user_list]

user_data = {}

for user_id, user_data_path in tqdm.tqdm(zip(user_list, data_path), total=len(user_list)):
    #   load user data
    user_data[user_id] = User(user_id, user_data_path, gt_df[user_id])

    # create segment features
    user_data[user_id].build_segment_features()


100%|██████████| 10/10 [00:05<00:00,  1.81it/s]


In [8]:
import matplotlib.pyplot as plt

all_test_segments = [user_data[user_id].segment_df_test.select_dtypes("number") for user_id in user_list]
all_test_segments_gt = [user_data[user_id].user_anomaly_gt for user_id in user_list]

all_test_segments = pd.concat(all_test_segments).reset_index(drop=True)
all_test_segments["anomaly"] = pd.concat(all_test_segments_gt).reset_index(drop=True)


corr = all_test_segments.corr()

In [9]:
corr

,unique_cmds,longest_same_cmd_sequence,cmds_not_in_train,starts_with_dotfile_cmds,single_chars_cmds,two_chars_cmds,three_chars_cmds,four_chars_cmds,ends_with_dot_cmds,has_dot_in_middle_cmds,has_zip_cmds,has_mail_cmds,has_coding_cmds,has_db_cmds,has_ssh_cmds,has_numerics_cmds,anomaly
unique_cmds,1.000000,-0.222672,0.189688,0.022948,0.413126,0.803381,0.605625,0.605625,NaN,0.017609,0.063597,0.444255,-0.044612,NaN,0.017942,0.351530,0.119453
longest_same_cmd_sequence,-0.222672,1.000000,-0.035606,-0.166376,0.007597,-0.215548,-0.249137,-0.249137,NaN,0.031985,-0.082661,-0.164555,-0.176382,NaN,-0.016682,-0.072231,-0.099179
cmds_not_in_train,0.189688,-0.035606,1.000000,0.008807,-0.043439,0.118965,0.265499,0.265499,NaN,0.232811,-0.046816,0.153882,0.064995,NaN,0.143348,0.230387,0.568771
starts_with_dotfile_cmds,0.022948,-0.166376,0.008807,1.000000,-0.078272,-0.141681,0.019067,0.019067,NaN,0.502096,0.053722,0.066548,0.077508,NaN,-0.018175,-0.024063,-0.045283
single_chars_cmds,0.413126,0.007597,-0.043439,-0.078272,1.000000,0.296298,0.116747,0.116747,NaN,-0.087854,-0.065240,0.259642,-0.098017,NaN,0.011035,0.257994,-0.055210
two_chars_cmds,0.803381,-0.215548,0.118965,-0.141681,0.296298,1.000000,0.559791,0.559791,NaN,-0.149182,0.061807,0.239647,0.178583,NaN,-0.009157,0.189438,0.080631
three_chars_cmds,0.605625,-0.249137,0.265499,0.019067,0.116747,0.559791,1.000000,1.000000,NaN,0.012823,0.138097,0.235099,0.501312,NaN,0.005804,0.471993,0.049610
four_chars_cmds,0.605625,-0.249137,0.265499,0.019067,0.116747,0.559791,1.000000,1.000000,NaN,0.012823,0.138097,0.235099,0.501312,NaN,0.005804,0.471993,0.049610
ends_with_dot_cmds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
has_dot_in_middle_cmds,0.017609,0.031985,0.232811,0.502096,-0.087854,-0.149182,0.012823,0.012823,NaN,1.000000,-0.017342,0.041446,-0.011945,NaN,-0.026949,0.002935,0.144612


,unique_cmds,longest_same_cmd_sequence,cmds_not_in_train,starts_with_dotfile_cmds,single_chars_cmds,two_chars_cmds,three_chars_cmds,four_chars_cmds,ends_with_dot_cmds,has_dot_in_middle_cmds,has_zip_cmds,has_mail_cmds,has_coding_cmds,has_db_cmds,has_ssh_cmds,has_numerics_cmds,anomaly
0,15,87,0,0,0,6,2,2,0,0,0,0,0,0,0,0,0.0
1,34,16,0,0,0,6,5,5,0,0,0,3,0,0,0,0,0.0
2,34,24,0,1,0,3,5,5,0,1,0,1,0,0,0,1,0.0
3,22,10,0,0,0,3,2,2,0,0,0,2,0,0,0,0,0.0
4,48,19,0,0,0,6,6,6,0,0,0,2,0,0,0,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,15,21,0,0,0,3,2,2,0,0,0,0,0,0,0,0,0.0
996,15,20,0,0,0,3,2,2,0,0,0,0,0,0,0,0,0.0
997,15,16,0,0,0,3,2,2,0,0,0,0,0,0,0,0,0.0
998,23,16,0,0,0,5,3,3,0,0,0,1,0,0,0,0,0.0


In [10]:
# import plotly.express as px
# fig = px.imshow(corr)
# fig.show()

In [11]:
''' first outlier detection algo '''
import itertools
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import precision_score, recall_score
from eval.metrics import detection_metrics


user_scores = {}

for user_id in tqdm.tqdm(user_list):

    anomaly_ground_truth = user_data[user_id].user_anomaly_gt.values.astype(int) # test set only
    anomaly_ground_truth = np.array([1 if x == 0 else -1 for x in anomaly_ground_truth])

    #   each detector is trained per user
    detector = LocalOutlierFactor(novelty=True, leaf_size=5)

    dummy_user_train_data = pd.get_dummies(user_data[user_id].segment_df_train)
    dummy_user_test_data = pd.get_dummies(user_data[user_id].segment_df_test)

    detector.fit(dummy_user_train_data)

    predicted_anomalies = detector.predict(dummy_user_test_data)

    user_scores[user_id] = detection_metrics(predicted_anomalies, anomaly_ground_truth)

overall_scores = {}
for score_type in ["detection_score" ,"precision_score", "recall_score"]:
    overall_scores[score_type] = np.mean([user_scores[user_id][score_type] for user_id in user_list])

100%|██████████| 10/10 [00:00<00:00, 27.35it/s]


In [16]:
overall_scores

{'detection_score': 0.37,
 'precision_score': 0.8321000000000002,
 'recall_score': 0.9267}

In [13]:
# from umap import UMAP
#
# reducer = UMAP()
# S_train_low_dim = reducer.fit_transform(train_feature_df)
# S_dev_low_dim = reducer.transform(dev_feature_df)

In [14]:
# detector = LocalOutlierFactor(novelty=True)
#
# detector.fit(S_train_low_dim)
#
# low_dim_predicted_anomalies = detector.predict(S_dev_low_dim)

In [15]:

import plotly.express as px

viz_df_train = pd.DataFrame({
    "x": S_train_low_dim[:, 0],
    "y": S_train_low_dim[:, 1],
    "user": TRAIN_SEGMENT_COUNT * [user_id],
    "set": TRAIN_SEGMENT_COUNT * ["train"]
})

viz_df_test = pd.DataFrame({
    "x": S_dev_low_dim[:, 0],
    "y": S_dev_low_dim[:, 1],
    "user": TEST_SEGMENT_COUNT * [user_id],
    "set": TEST_SEGMENT_COUNT * ["test"]
})


viz_df = pd.concat([viz_df_train, viz_df_test], axis=0)
viz_df["anomaly"] = gt_df.T[user_id].values
viz_df["anomaly"] = viz_df["anomaly"].astype(int)


NameError: name 'S_train_low_dim' is not defined

In [ ]:
# import plotly.io as pio
# pio.renderers.default='notebook'
#
px.scatter(viz_df, x="x", y="y", color="anomaly")